In [ ]:
import os
os.environ['KAGGLE_USERNAME'] =
os.environ['KAGGLE_KEY'] =
!kaggle datasets download -d saurabhshahane/fake-news-classification

Dataset URL: https://www.kaggle.com/datasets/saurabhshahane/fake-news-classification
License(s): Attribution 4.0 International (CC BY 4.0)


In [ ]:
!unzip /content/fake-news-classification.zip

Archive:  /content/fake-news-classification.zip
  inflating: WELFake_Dataset.csv     


In [ ]:
import pandas as pd
df = pd.read_csv('/content/WELFake_Dataset.csv')

In [ ]:
df.head()

,Unnamed: 0,title,text,label,content
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1,LAW ENFORCEMENT ON HIGH ALERT Following Threat...
1,1,,Did they post their votes for Hillary already?,1,Did they post their votes for Hillary already?
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0,"Bobby Jindal, raised Hindu, uses story of Chri..."
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1,SATAN 2: Russia unvelis an image of its terrif...


In [ ]:
df['content'] = df['title']+' '+df['text']

In [ ]:
df['content'][1]

' Did they post their votes for Hillary already?'

##Check null values

In [ ]:
df.isnull().sum()

,0
Unnamed: 0,0
title,558
text,39
label,0


In [ ]:
df = df.fillna('')

In [ ]:
df.isnull().sum()

,0
Unnamed: 0,0
title,0
text,0
label,0


##Preprocessing of text

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# remove punctuations
import string
exclude = string.punctuation
# transform characters in a string according to str.maketrans table
def remove_punc(text):
  return text.translate(str.maketrans('','',exclude))

In [ ]:
df['content'] = df['content'].apply(remove_punc)

In [ ]:
# remove stopwords (meaningless words)
from nltk.corpus import stopwords

# Load stopwords once as a set for O(1) lookup
STOPWORDS = set(stopwords.words('english'))

def remove_stopwords(text):
    # Split text and filter out stopwords in one pass
    return " ".join(word for word in text.split() if word not in STOPWORDS)

In [ ]:
df['content'] = df['content'].apply(remove_stopwords)

In [ ]:
# remove non-alphabetic
import re
def remove_non_alpha(text):
    # Replace non-alphabetical characters with space
    return re.sub('[^a-zA-Z]', ' ', text)

In [ ]:
df['content'] = df['content'].apply(remove_non_alpha)

In [ ]:
# convert lower
df['content'] = df['content'].apply(lambda x: x.lower())

In [ ]:
# word stemming
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
def stem_words(text):
    return " ".join([ps.stem(word) for word in text.split()])

In [ ]:
df['content'] = df['content'].apply(stem_words)

In [ ]:
df['content']

,content
0,law enforc on high alert follow threat against...
1,did post vote hillari alreadi
2,unbeliev obama s attorney gener say most charl...
3,bobbi jindal rais hindu use stori christian co...
4,satan russia unv imag terrifi new supernuk wes...
...,...
72129,russian steal research trump hack us democrat ...
72130,watch giuliani demand that democrat apolog for...
72131,migrant refus to leav train at refuge camp in ...
72132,trump tussl give unpopular mexican leader much...


## Model Training

In [ ]:
X = df['content'].values
Y = df['label'].values

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train, Y_train)

LogisticRegression()

In [ ]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9686519832949209


In [ ]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9564011922090525


In [ ]:
X_new = X_test[2]
prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[1]
The news is Fake
